In [2]:
import json
import pandas as pd
from datetime import datetime


In [62]:
LIST_OF_CHANNEL_IDS = [
"UCsDmESjqNPukDmVnuneLrqw",
"UCY6Ij8zOds0WJEeqCLOnqOQ",
"UClN24S5-fc7LD0JJW0FdGWw",
"UC3iNdSV_RQU7DHTGpqEW96w",
"UC_52i6mlrzOZs_NEuSepzAg",
"UCr3cBLTYmIK9kY0F_OdFWFQ",
"UCV6g95OBbVtFmN9uiJzkFqQ",
"UCgCKYs56-LKEPGQ99DzqQOg",
"UCOwxx9VnEnlFKt5EB70KTzQ",
"UC2Ds30pkifFVD0CE08wF50g",
"UC7UGbBVrqLlq6CRxWWUmyKw",
"UCn1XB-jvmd9fXMzhiA6IR0w",
"UCFeqAfEuKm7lIg2ddQzh61A",
"UCAlxwHEm1bWWYoe5VCSQYEg",
"UCz7iJPVTBGX6DNO1RNI2Fcg",
"UCOAUz4d1NUNxSHh_SOJtdGQ",
"UCGGTAB19HlHEWPwwmxHsEKA",
"UCOsATJw-IZgqGT8MFrHjKGg",
"UCsKVP_4zQ877TEiH_Ih5yDQ",
"UCOinp9hALPUcZeoL3Kmaviw",
"UCGwu0nbY2wSkW8N-cghnLpA",
"UCu6v4AdYxVH5fhfq9mi5llA",
"UCj74rJ9Lgl3WTngq675wxKg",
"UCNIuvl7V8zACPpTmmNIqP2A",
"UCo8bcnLyZH8tBIH9V1mLgqQ",
"UCR0O-1cvuPNxDosvSDLpWHg",
"UCvUmwreRrbxeR1mbmojj8fg",
"UCUKi4zY5ETSqrKAjTBgjM-g",
"UCDQBZcjYKP1J1Nu-Y0_D37Q",
"UCQ9HvHH-KRYHI5ynj2kbLwQ",
"UCRfg0SWjIHm_h95e4V8X5og",
"UCWPB0WpnMIy-g7zncwIhvQg",
"UCC-slLJZ4p4HOznMUcFn_2g"]

In [149]:
df = pd.read_json (r'channelListResponse.json')
REFERENCE_UTC_NOW= datetime(2021, 12, 12, 5, 2, 59, 166993)
total_df= pd.DataFrame()

for index, row in df.iterrows():
    items=df['items'][index][0]
    snippet = items['snippet']

    title=snippet['title']
    channelId=items['id']
    publishedAt=snippet['publishedAt']
    description=snippet['description']
    viewCount=items['statistics']['viewCount']
    subscriberCount=items['statistics']['subscriberCount']
    videoCount=items['statistics']['videoCount']
    daysLive = pd.Timedelta(REFERENCE_UTC_NOW - datetime.strptime(publishedAt, '%Y-%m-%d'+'T'+'%H:%M:%S'+'Z')).round('d')
    data = {'title':title, 
    'channelId': channelId, 
    'daysLiveAsOfLaunch': daysLive,
    'description': description,
    'viewCount': int(viewCount),
    'subscriberCount': int(subscriberCount),
    'videoCount': int(videoCount),
    'subscribersPerDay' : int(subscriberCount/daysLive),
    'viewCountPerDay' : int(viewCount/daysLive)
    }
    df_generated = pd.DataFrame(data, index=[index])
    total_df= total_df.append(df_generated)
print(total_df['videoCount'].sum(), " videos to process in the next cell")
total_df.head()
#important. date ref is 12/11 because this is when I pulled this data. 

3236  videos to process in the next cell


,title,channelId,daysLiveAsOfLaunch,description,viewCount,subscriberCount,videoCount,subscribersPerDay,viewCountPerDay
0,Alex Clark,UCsDmESjqNPukDmVnuneLrqw,4392 days,Funny vids about my life. I'm thankful for th...,471903125,4280000,317,0,0
1,Alex Meyers,UCY6Ij8zOds0WJEeqCLOnqOQ,2348 days,I make cartoons and sometimes video essays abo...,472627239,2870000,256,0,0
2,WowRightMeow,UClN24S5-fc7LD0JJW0FdGWw,1946 days,"Yo! I'm Jamie, a dorky human from Wales. Did a...",29732357,537000,33,0,0
3,Andrei Terbea,UC3iNdSV_RQU7DHTGpqEW96w,3147 days,Doing animated commentary and solving crimes w...,292193694,2860000,136,0,0
4,Billy But Better,UC_52i6mlrzOZs_NEuSepzAg,2805 days,"Yes, it's a foot.\n\nSocial Medias:\nTwitter -...",12166120,260000,23,0,0


In [85]:
df = pd.read_json (r'searchListResponse.json')
channelIdToListOfVids = {} #channelId: []vids
backend_purposes = {}


#initialize ultimate dict
for channelId in LIST_OF_CHANNEL_IDS:
    channelIdToListOfVids[channelId] = []
    backend_purposes[channelId] = []

#youtuber
#0~9ish: items
#items: 0~46ish
test = df.loc[0, LIST_OF_CHANNEL_IDS[0]]
for index, midList in df.iterrows():  
    listOfLists = df.loc[index, LIST_OF_CHANNEL_IDS[index]]
    for indList in listOfLists:
        for item in indList['items']:
            channelId = item['snippet']['channelId']
            if 'videoId' in item['id'].keys():
                snippet=item['snippet']
                vidId = item['id']['videoId']
                vidPublishedAt = snippet['publishedAt']
                publishTime = snippet['publishTime']
                vidTitle = snippet['title']
                vidDesc = snippet['description']

                vidInfo = {'vidId' : vidId,
                'vidPublishedAt': vidPublishedAt,
                'publishTime': publishTime,
                'vidTitle': vidTitle,
                'vidDescription': vidDesc}
                backend_purposes[channelId].append(vidId)
                channelIdToListOfVids[channelId].append(vidInfo)

count= 0
for i in backend_purposes.keys():
    count += len(backend_purposes[i])
print(count, " videos detected")

print(len(channelIdToListOfVids.keys()), " keys detected")
vidCont=0
for i in channelIdToListOfVids.keys():
    vidCont+=len(channelIdToListOfVids[i])
print(count, " videos detected")
channelIdToListOfVids

3223  videos detected
33  keys detected
3223  videos detected


{'UCsDmESjqNPukDmVnuneLrqw': [{'vidId': 'beSntf_CNKA',
   'vidPublishedAt': '2019-09-18T21:07:40Z',
   'publishTime': '2019-09-18T21:07:40Z',
   'vidTitle': 'Mean Girls Got Me A Wife (Part 3)',
   'vidDescription': "Mean Girls is a great movie.... and my wife agreed. Huion Contest: http://www.itsalexclark.com/huion Pam's channel: ..."},
  {'vidId': '9p763a_DGHo',
   'vidPublishedAt': '2016-09-21T12:12:34Z',
   'publishTime': '2016-09-21T12:12:34Z',
   'vidTitle': '&quot;Bonding&quot; With Uncle',
   'vidDescription': "Uncle thinks he's the only one with a gun... Now you can mail me stuff: Alex Clark P.O. Box 192022 Los Angeles, CA 90019 I'm ..."},
  {'vidId': 'rYfEG24vVx4',
   'vidPublishedAt': '2016-08-25T16:16:11Z',
   'publishTime': '2016-08-25T16:16:11Z',
   'vidTitle': '7 Day Vegan Challenge Baby (Solves All Your Problems)',
   'vidDescription': "Thanks Jaiden :) Now you can mail me stuff: Alex Clark P.O. Box 192022 Los Angeles, CA 90019 I'm Alex Clark and I make ..."},
  {'vidId'

In [94]:
listOfVideoIds=[]
for key, value in backend_purposes.items():
    listOfVideoIds+=value
with open('listOfVideoIds.json', 'w') as outfile:
    json.dump(listOfVideoIds, outfile)

In [100]:
df = pd.read_json (r'videoListResponse.json')
matching_vidId_to_info={}
count = 0
for index, indList in df.iterrows():
    items = indList['items']
    for item in items:
        count+=1
        vidId = item['id']
        contentDetails= item['contentDetails']
        duration= contentDetails['duration']
        statistics = item['statistics']
        viewCount = statistics['viewCount']
        likeCount = statistics.get('likeCount')
        commentCount = statistics.get('commentCount')
        topicCategories = []
        if 'topicDetails' in item.keys():
            topicCategories = item['topicDetails']['topicCategories'] #list
        matching_vidId_to_info[vidId] = {'duration': duration,
                        'viewCount': viewCount,
                        'likeCount': likeCount,
                        'commentCount': commentCount,
                        'topicCategories': topicCategories}
print(count)
len(matching_vidId_to_info)


3222


3208

In [105]:
#deepcopy
channelIdToListOfVids_copy = json.loads(json.dumps(channelIdToListOfVids))

for channelId in LIST_OF_CHANNEL_IDS:
    listOfVideosForEachChannel = channelIdToListOfVids_copy[channelId]
    for video in listOfVideosForEachChannel:
        additionalVideoMetadata= matching_vidId_to_info.get(video['vidId'])
        if additionalVideoMetadata:
            video.update(additionalVideoMetadata)
        else:
            print(video['vidId'])

print(sum(len(channelIdToListOfVids_copy[channelId]) for channelId in LIST_OF_CHANNEL_IDS))
#some of these vids are private, duplicates. 

GenGZzJzYCM
3223
